In [ ]:
from pathlib import Path

import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.figure import Figure

from analytics.app.data.load import list_pipelines
from analytics.app.data.transform import dfs_models_and_evals, logs_dataframe
from analytics.plotting.common.common import init_plot
from analytics.plotting.common.font import setup_font
from analytics.plotting.common.heatmap import build_heatmap
from analytics.plotting.common.save import save_plot

%load_ext autoreload
%autoreload 2

In [ ]:
pipelines_dir = Path(
    "/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/arxiv/10_baselines_time"
)
assert pipelines_dir.exists()

In [ ]:
pipelines = list_pipelines(pipelines_dir)
max_pipeline_id = max(pipelines.keys())
pipelines

In [ ]:
from analytics.app.data.load import load_pipeline_logs

pipeline_logs = {p_id: load_pipeline_logs(p_id, pipelines_dir) for (p_id, (_, p_path)) in pipelines.items()}

In [ ]:
# mode:
pipeline_id = 267

# doesn't do anything unless include_composite_model = True
composite_model_variant = "currently_active_model"

dataset_id = "arxiv_kaggle_test"
eval_handler = "periodic-current"
metrics = [
    "Accuracy",
    "F1-micro",
    "F1-macro",
    "F1-weighted",
    "Top-10-Accuracy",
    "Top-5-Accuracy",
    "Top-2-Accuracy",
]
include_composite_model = False

# Wrangle data

In [ ]:
pipeline_log = pipeline_logs[pipeline_id]
pipeline_ref = f"{pipeline_id}".zfill(len(str(max_pipeline_id))) + f" - {pipelines[pipeline_id][0]}"

df_all = logs_dataframe(pipeline_log, pipeline_ref)

df_logs_models, _, df_eval_single = dfs_models_and_evals(
    # subtracting would interfere with yearbook patching
    pipeline_log,
    df_all["sample_time"].max(),
    pipeline_ref,
)

df_adjusted = df_eval_single

df_adjusted = df_adjusted[
    (df_adjusted["dataset_id"] == dataset_id)
    & (df_adjusted["eval_handler"] == eval_handler)
    & (df_adjusted["metric"].isin(metrics))
]

# in percent (0-100)
df_adjusted["value"] = df_adjusted["value"] * 100
df_adjusted

In [ ]:
df_adjusted = df_adjusted.sort_values(by=["interval_center"])
df_adjusted["interval_center"] = df_adjusted["interval_center"].dt.to_period("M")
df_adjusted

In [ ]:
# Add composite model

assert df_adjusted["pipeline_ref"].nunique() <= 1
# add the pipeline time series which is the performance of different models stitched together dep.
# w.r.t which model was active
pipeline_composite_model = df_adjusted[df_adjusted[composite_model_variant]]
pipeline_composite_model["model_idx"] = 0
pipeline_composite_model["id_model"] = 0

label_map = {k: f"{k}" for k, v in df_adjusted[["model_idx", "id_model"]].values}
label_map[0] = "Pipeline composite model"

if include_composite_model:
    df_adjusted = pd.concat([pipeline_composite_model, df_adjusted])
else:
    df_adjusted["model_idx"] = df_adjusted["model_idx"]

# Create Plot

In [ ]:
def plot_heatmap_grid(
    data: dict[tuple[int, str], dict[tuple[int, str], pd.DataFrame]],
    cbar_label: str,
    # Define vmin and vmax for the color scale to be consistent across heatmaps
    vmin: float = 0,
    vmax: float = 0.3,
    nrows: int = 4,
    ncols: int = 4,
    cbar: bool = False,
    single_cbar: bool = False,
    height_factor: float = 1.2,
    width_factor: float = 1.0,
    x_space_factor: float = 1,
    y_space_factor: float = 1,
    grid_alpha: float = 0.0,
) -> Figure:
    init_plot()
    setup_font(small_label=True, small_title=True)

    double_fig_width = 10
    double_fig_height = 3.5
    fig, axs = plt.subplots(
        nrows=nrows,
        ncols=ncols,
        edgecolor="black",
        frameon=True,
        figsize=(double_fig_width * width_factor, 2.2 * double_fig_height * height_factor),
        dpi=450,
        squeeze=True,
    )

    for x, row_key in enumerate(data):
        (row, row_title) = row_key
        for y, col_key in enumerate(data[row_key]):
            (col, col_title) = col_key
            title, cell_data = data[row_key][col_key]
            ax = axs[x, y] if nrows > 1 else axs[y]
            # print([(i, period.to_timestamp().strftime('%b %Y')) for i, period in list(enumerate(cell_data.columns))[::1]])
            # available:
            # [(0, 'Apr 1995'), (1, 'Oct 1995'), (2, 'Mar 1996'), (3, 'Sep 1996'), (4, 'Mar 1997'), (5, 'Sep 1997'), (6, 'Mar 1998'), (7, 'Sep 1998'), (8, 'Mar 1999'), (9, 'Sep 1999'), (10, 'Mar 2000'), (11, 'Sep 2000'), (12, 'Mar 2001'), (13, 'Sep 2001'), (14, 'Mar 2002'), (15, 'Sep 2002'), (16, 'Mar 2003'), (17, 'Sep 2003'), (18, 'Mar 2004'), (19, 'Sep 2004'), (20, 'Mar 2005'), (21, 'Sep 2005'), (22, 'Mar 2006'), (23, 'Sep 2006'), (24, 'Mar 2007'), (25, 'Sep 2007'), (26, 'Mar 2008'), (27, 'Sep 2008'), (28, 'Mar 2009'), (29, 'Sep 2009'), (30, 'Mar 2010'), (31, 'Sep 2010'), (32, 'Mar 2011'), (33, 'Sep 2011'), (34, 'Mar 2012'), (35, 'Sep 2012'), (36, 'Mar 2013'), (37, 'Sep 2013'), (38, 'Mar 2014'), (39, 'Sep 2014'), (40, 'Mar 2015'), (41, 'Sep 2015'), (42, 'Mar 2016'), (43, 'Sep 2016'), (44, 'Mar 2017'), (45, 'Sep 2017'), (46, 'Mar 2018'), (47, 'Sep 2018'), (48, 'Mar 2019'), (49, 'Sep 2019'), (50, 'Mar 2020'), (51, 'Aug 2020'), (52, 'Feb 2021'), (53, 'Aug 2021'), (54, 'Feb 2022'), (55, 'Aug 2022'), (56, 'Feb 2023'), (57, 'Aug 2023'), (58, 'Feb 2024')]

            # print([(i, period.to_timestamp().strftime('%b %Y')) for i, period in list(enumerate(cell_data.index))[::1]])
            # [(0, 'Jul 1995'), (1, 'Dec 1995'), (2, 'Jun 1996'), (3, 'Dec 1996'), (4, 'Jun 1997'), (5, 'Dec 1997'), (6, 'Jun 1998'), (7, 'Dec 1998'), (8, 'Jun 1999'), (9, 'Dec 1999'), (10, 'Jun 2000'), (11, 'Dec 2000'), (12, 'Jun 2001'), (13, 'Dec 2001'), (14, 'Jun 2002'), (15, 'Dec 2002'), (16, 'Jun 2003'), (17, 'Dec 2003'), (18, 'Jun 2004'), (19, 'Dec 2004'), (20, 'Jun 2005'), (21, 'Dec 2005'), (22, 'Jun 2006'), (23, 'Dec 2006'), (24, 'Jun 2007'), (25, 'Dec 2007'), (26, 'Jun 2008'), (27, 'Dec 2008'), (28, 'Jun 2009'), (29, 'Dec 2009'), (30, 'Jun 2010'), (31, 'Dec 2010'), (32, 'Jun 2011'), (33, 'Dec 2011'), (34, 'Jun 2012'), (35, 'Dec 2012'), (36, 'Jun 2013'), (37, 'Dec 2013'), (38, 'Jun 2014'), (39, 'Dec 2014'), (40, 'Jun 2015'), (41, 'Dec 2015'), (42, 'Jun 2016'), (43, 'Dec 2016'), (44, 'Jun 2017'), (45, 'Dec 2017'), (46, 'Jun 2018'), (47, 'Dec 2018'), (48, 'Jun 2019'), (49, 'Dec 2019'), (50, 'May 2020'), (51, 'Nov 2020'), (52, 'May 2021'), (53, 'Nov 2021'), (54, 'May 2022'), (55, 'Nov 2022'), (56, 'May 2023'), (57, 'Nov 2023'), (58, 'May 2024')]

            _ = build_heatmap(
                cell_data,
                x_custom_ticks=[
                    (i, f"{period.to_timestamp().strftime('%b %Y')}".replace(" ", "\n"))
                    for i, period in list(enumerate(cell_data.columns))[::1]
                    if period in [pd.Period("Mar 2000"), pd.Period("Mar 2009"), pd.Period("Mar 2020")]
                ],
                y_custom_ticks=[
                    (i, f"{period.to_timestamp().strftime('%b %Y')}".replace(" ", "\n"))
                    for i, period in list(enumerate(cell_data.index))[::1]
                    if period in [pd.Period("Jun 2000"), pd.Period("Jun 2009"), pd.Period("May 2020")]
                ],
                x_label=col_title,
                y_label=row_title,
                reverse_col=True,
                # x_label,
                # color_label = "MMD",  # TODO
                title_label=title,
                target_ax=ax,
                square=False,
                width_factor=width_factor,
                height_factor=height_factor,
                cbar=single_cbar,
                vmin=vmin,
                vmax=vmax,
                grid_alpha=grid_alpha,
            )
            ax.label_outer()  # Remove labels for inner plots, keep only on outer

    if cbar:
        cbar_ax = fig.add_axes([1, 0.25, 0.02, 0.6])  # Adjust the colorbar position
        custom_cbar = fig.colorbar(ax.collections[0], cax=cbar_ax)  # use last printed axis
        custom_cbar.set_label(cbar_label)  # Set your custom label here

    plt.subplots_adjust(wspace=0.1 * x_space_factor, hspace=0.1 * y_space_factor)
    return fig

In [ ]:
df_train_end_years_per_model = df_logs_models[["model_idx", "real_train_end"]]
df_train_end_years_per_model["real_train_end"] = df_train_end_years_per_model["real_train_end"].dt.to_period("M")
df_train_end_years_per_model

In [ ]:
df_merged = df_adjusted.merge(df_train_end_years_per_model, on="model_idx", how="left")
df_merged

In [ ]:
def generate_heatmap_data_for_handler(data: pd.DataFrame, metric: str) -> pd.DataFrame:
    # build heatmap matrix dataframe:
    data_filtered = data[data["metric"] == metric]
    pt_data = data_filtered.pivot(index=["real_train_end"], columns="interval_center", values="value")
    return pt_data

In [ ]:
plot_content = {
    (0, "Trained up to"): {
        (0, "Evaluation Year"): (
            "Accuracy",
            generate_heatmap_data_for_handler(
                df_merged, "Accuracy"
            ),  # almost identical to F1-micro and F1-weighted; macro is broken
        ),
        (1, "Evaluation Year"): ("Top-2-Accuracy", generate_heatmap_data_for_handler(df_merged, "Top-2-Accuracy")),
        (2, "Evaluation Year"): ("Top-5-Accuracy", generate_heatmap_data_for_handler(df_merged, "Top-5-Accuracy")),
        (3, "Evaluation Year"): ("Top-10-Accuracy", generate_heatmap_data_for_handler(df_merged, "Top-10-Accuracy")),
    }
}

# find vmin and vmax
vmin = 1
vmax = 0
for row_key in plot_content:
    for col_key in plot_content[row_key]:
        (_, cell_data) = plot_content[row_key][col_key]
        vmin = min(vmin, cell_data.min().min())
        vmax = max(vmax, cell_data.max().max())
print(vmin, vmax)

fig = plot_heatmap_grid(
    data=plot_content,
    cbar_label="[Metric] %",
    nrows=1,
    ncols=4,
    vmin=vmin,
    vmax=vmax,
    cbar=True,
    single_cbar=False,
    width_factor=1,
    height_factor=0.48,
    grid_alpha=0.5,
)

# TODO: we have already seen the Accuracy plot in Offline eval chapter (with another color scale though)
save_plot(fig, "evaluation_metrics_arxiv")